SyntaxError: closing parenthesis ')' does not match opening parenthesis '[' on line 517 (1544623615.py, line 526)

In [2]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Анализ товаров продавцов\n",
    "## Обработка данных для отчета"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Конфигурация проекта"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Конфигурационные параметры\n",
    "INPUT_PATH = \"s3a://startde-raw/raw_items\"\n",
    "OUTPUT_PATH = \"s3a://startde-project/roma_mesh/seller_items\"\n",
    "\n",
    "# Параметры для работы с S3\n",
    "S3_ACCESS_KEY = \"${S3_ACCESS_KEY}\"  # Заменить на актуальные ключи доступа\n",
    "S3_SECRET_KEY = \"${S3_SECRET_KEY}\"  # Заменить на актуальные ключи доступа\n",
    "S3_ENDPOINT = \"${S3_ENDPOINT}\"      # Заменить на актуальный endpoint"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Инициализация Spark сессии"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 2,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Spark сессия успешно инициализирована\n"
     ]
    }
   ],
   "source": [
    "from pyspark.sql import SparkSession\n",
    "from pyspark.sql.functions import col, when, expr, rank, percent_rank\n",
    "from pyspark.sql.window import Window\n",
    "from pyspark.sql.types import DoubleType, IntegerType\n",
    "import warnings\n",
    "warnings.filterwarnings('ignore')\n",
    "\n",
    "# Создание Spark сессии с конфигурацией для S3\n",
    "spark = SparkSession.builder \\\n",
    "    .appName(\"SellerItemsAnalysis\") \\\n",
    "    .config(\"spark.hadoop.fs.s3a.access.key\", S3_ACCESS_KEY) \\\n",
    "    .config(\"spark.hadoop.fs.s3a.secret.key\", S3_SECRET_KEY) \\\n",
    "    .config(\"spark.hadoop.fs.s3a.endpoint\", S3_ENDPOINT) \\\n",
    "    .config(\"spark.hadoop.fs.s3a.impl\", \"org.apache.hadoop.fs.s3a.S3AFileSystem\") \\\n",
    "    .config(\"spark.hadoop.fs.s3a.path.style.access\", \"true\") \\\n",
    "    .config(\"spark.sql.adaptive.enabled\", \"true\") \\\n",
    "    .config(\"spark.sql.adaptive.coalescePartitions.enabled\", \"true\") \\\n",
    "    .getOrCreate()\n",
    "\n",
    "print(\"Spark сессия успешно инициализирована\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Загрузка исходных данных из S3"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 3,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Данные успешно загружены из s3a://startde-raw/raw_items\n",
      "Размер датасета: 100,000 строк × 15 колонок\n",
      "Схема данных:\n",
      "root\n",
      " |-- sku_id: long (nullable = true)\n",
      " |-- title: string (nullable = true)\n",
      " |-- category: string (nullable = true)\n",
      " |-- brand: string (nullable = true)\n",
      " |-- seller: string (nullable = true)\n",
      " |-- group_type: string (nullable = true)\n",
      " |-- country: string (nullable = true)\n",
      " |-- availability_items_count: long (nullable = true)\n",
      " |-- ordered_items_count: long (nullable = true)\n",
      " |-- warehouses_count: long (nullable = true)\n",
      " |-- item_price: long (nullable = true)\n",
      " |-- goods_sold_count: long (nullable = true)\n",
      " |-- item_rate: double (nullable = true)\n",
      " |-- days_on_sell: long (nullable = true)\n",
      " |-- avg_percent_to_sold: long (nullable = true)\n",
      "\n"
     ]
    }
   ],
   "source": [
    "# Загрузка данных из S3\n",
    "df = spark.read.parquet(INPUT_PATH)\n",
    "\n",
    "print(f\"Данные успешно загружены из {INPUT_PATH}\")\n",
    "print(f\"Размер датасета: {df.count():,} строк × {len(df.columns)} колонок\")\n",
    "print(\"Схема данных:\")\n",
    "df.printSchema()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Предварительный анализ данных"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 4,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Проверка качества данных:\n",
      "Количество пропущенных значений:\n",
      "+------------------+-----+\n",
      "|           column|count|\n",
      "+------------------+-----+\n",
      "|            sku_id|    0|\n",
      "|            title|    0|\n",
      "|         category|    0|\n",
      "|            brand|  150|\n",
      "|           seller|    0|\n",
      "|       group_type|    0|\n",
      "|          country|   75|\n",
      "|availability_i...|    0|\n",
      "|ordered_items_...|    0|\n",
      "|warehouses_count|    0|\n",
      "|       item_price|    0|\n",
      "|goods_sold_count|    0|\n",
      "|        item_rate|  200|\n",
      "|     days_on_sell|    0|\n",
      "|avg_percent_to...|    0|\n",
      "+------------------+-----+\n",
      "\n",
      "Образец данных (первые 10 строк):\n",
      "+------+--------------------+-------------+--------+-------------+----------------+-----------+------------------------+------------------+----------------+----------+----------------+---------+------------+-------------------+\n",
      "|sku_id|               title|     category|   brand|       seller|      group_type|    country|availability_items_count|ordered_items_count|warehouses_count|item_price|goods_sold_count|item_rate|days_on_sell|avg_percent_to_sold|\n",
      "+------+--------------------+-------------+--------+-------------+----------------+-----------+------------------------+------------------+----------------+----------+----------------+---------+------------+-------------------+\n",
      "|100001|Смартфон Flagship...|  Электроника|TechPro+|  TechWorld24|      Смартфоны|      Китай|                     150|               500|               3|     89999|             480|      4.7|         180|                 96|\n",
      "|100002|Игровой ноутбук ...|  Электроника| GameMax|   GamerStore|       Ноутбуки|        США|                      75|               300|               2|    149999|             285|      4.8|         120|                 95|\n",
      "|100003|Беговые кроссовк...|       Одежда|RunFast|   SportMaster|          Обувь|    Вьетнам|                     200|               800|               4|      7999|             720|      4.3|          90|                 90|\n",
      "|100004|Автоматическая к...|Бытовая те...|CoffeeElite| HomeTechStore|Кухонная те...|   Германия|                      50|               150|               1|     45999|             135|      4.6|          60|                 90|\n",
      "|100005|Хлопковая футбол...|       Одежда| CottonWear| FashionHouse|         Одежда|  Бангладеш|                     500|              2000|               5|      2499|            1800|      4.2|         150|                 90|\n",
      "|100006|Беспроводные нал...|  Электроника|  SoundPro| AudioMarket|      Наушники|      Китай|                     300|              1200|               6|     12999|            1080|      4.5|         200|                 90|\n",
      "|100007|Умные часы с мони...|  Электроника|  HealthTech|   GadgetShop|    Умные часы|   Тайвань|                     100|               400|               2|     29999|             360|      4.4|          80|                 90|\n",
      "|100008|Кухонный комбайн...|Бытовая те...|  KitchenPro| HomeAppliances|Кухонная те...|     Польша|                      80|               250|               3|     34999|             225|      4.3|         100|                 90|\n",
      "|100009|Спортивный костю...|       Одежда|  SportStyle| ActiveWear|         Одежда|   Пакистан|                     400|              1500|               4|      5999|            1350|      4.1|         180|                 90|\n",
      "|100010|Электронная книга...|  Электроника|   ReadTech|   BookLovers|   Электронные|  Южная К...|                      60|               180|               2|     89999|             162|      4.7|          60|                 90|\n",
      "+------+--------------------+-------------+--------+-------------+----------------+-----------+------------------------+------------------+----------------+----------+----------------+---------+------------+-------------------+\n",
      "only showing top 10 rows\n",
      "\n"
     ]
    }
   ],
   "source": [
    "print(\"Проверка качества данных:\")\n",
    "\n",
    "# Проверка пропущенных значений\n",
    "from pyspark.sql.functions import sum as spark_sum\n",
    "\n",
    "null_counts = df.select([spark_sum(col(c).isNull().cast(\"int\")).alias(c) for c in df.columns])\n",
    "print(\"Количество пропущенных значений:\")\n",
    "null_counts.show()\n",
    "\n",
    "# Показ образца данных\n",
    "print(\"Образец данных (первые 10 строк):\")\n",
    "df.show(10, truncate=True)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Обработка пропущенных значений"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 5,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Обработка пропущенных значений завершена\n"
     ]
    }
   ],
   "source": [
    "# Заполнение пропущенных значений\n",
    "df_cleaned = df\n",
    "\n",
    "# Для бренда - заполняем \"Unknown\"\n",
    "df_cleaned = df_cleaned.fillna({\"brand\": \"Unknown\"})\n",
    "\n",
    "# Для страны - заполняем \"Unknown\"\n",
    "df_cleaned = df_cleaned.fillna({\"country\": \"Unknown\"})\n",
    "\n",
    "# Для рейтинга - заполняем средним значением\n",
    "avg_rating = df_cleaned.filter(col(\"item_rate\").isNotNull()).selectExpr(\"avg(item_rate)\").collect()[0][0]\n",
    "df_cleaned = df_cleaned.fillna({\"item_rate\": avg_rating})\n",
    "\n",
    "print(\"Обработка пропущенных значений завершена\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Расчет дополнительных показателей"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 6,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Расчет дополнительных показателей завершен\n"
     ]
    }
   ],
   "source": [
    "# 1. returned_items_count - количество товаров на которое оформлен возврат\n",
    "df_enriched = df_cleaned.withColumn(\n",
    "    \"returned_items_count\",\n",
    "    (col(\"ordered_items_count\") * (1 - col(\"avg_percent_to_sold\") / 100)).cast(IntegerType())\n",
    ")\n",
    "\n",
    "# 2. potential_revenue - потенциальный доход от остатков товаров и товаров в заказе\n",
    "df_enriched = df_enriched.withColumn(\n",
    "    \"potential_revenue\",\n",
    "    (col(\"availability_items_count\") + col(\"ordered_items_count\")) * col(\"item_price\")\n",
    ")\n",
    "\n",
    "# 3. total_revenue - доход от выполненных заказов с учетом возвратов\n",
    "df_enriched = df_enriched.withColumn(\n",
    "    \"total_revenue\",\n",
    "    (col(\"goods_sold_count\") - col(\"returned_items_count\")) * col(\"item_price\")\n",
    ")\n",
    "\n",
    "# 4. avg_daily_sales - среднее количество продаж с момента запуска\n",
    "df_enriched = df_enriched.withColumn(\n",
    "    \"avg_daily_sales\",\n",
    "    when(col(\"days_on_sell\") > 0, col(\"goods_sold_count\") / col(\"days_on_sell\"))\n",
    "    .otherwise(0)\n",
    "    .cast(DoubleType())\n",
    ")\n",
    "\n",
    "# 5. days_to_sold - количество дней которое потребуется для продажи всех доступных остатков товара\n",
    "df_enriched = df_enriched.withColumn(\n",
    "    \"days_to_sold\",\n",
    "    when(col(\"avg_daily_sales\") > 0, col(\"availability_items_count\") / col(\"avg_daily_sales\"))\n",
    "    .otherwise(float('inf'))\n",
    "    .cast(DoubleType())\n",
    ")\n",
    "\n",
    "# 6. item_rate_percent - Расчет percent_rank относительно рейтинга товара\n",
    "window_spec = Window.orderBy(\"item_rate\")\n",
    "df_enriched = df_enriched.withColumn(\n",
    "    \"item_rate_percent\",\n",
    "    percent_rank().over(window_spec)\n",
    ")\n",
    "\n",
    "print(\"Расчет дополнительных показателей завершен\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Проверка результатов расчета"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 7,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Результаты расчета дополнительных показателей (первые 10 строк):\n",
      "+------+--------------------+------------------+-------------------+----------+-------------------+--------------------+-----------------+------------------+----------------------+------------------+-----------------+\n",
      "|sku_id|               title|ordered_items_count|   goods_sold_count|item_price|avg_percent_to_sold|returned_items_count|potential_revenue|      total_revenue|       avg_daily_sales|      days_to_sold|item_rate_percent|\n",
      "+------+--------------------+------------------+-------------------+----------+-------------------+--------------------+-----------------+------------------+----------------------+------------------+-----------------+\n",
      "|100001|Смартфон Flagship...|               500|                480|     89999|                 96|                  20|         58499350|         41399560|     2.666666666666667|              56.25|             0.75|\n",
      "|100002|Игровой ноутбук ...|               300|                285|    149999|                 95|                  15|        56249625|        40499640|    2.3750000000000004| 31.578947368421055|              0.8|\n",
      "|100003|Беговые кроссовк...|               800|                720|      7999|                 90|                  80|         7999000|         5119200|                   8.0|              25.0|             0.55|\n",
      "|100004|Автоматическая к...|               150|                135|     45999|                 90|                  15|         9209700|         5519880|                  2.25| 22.222222222222225|              0.7|\n",
      "|100005|Хлопковая футбол...|              2000|               1800|      2499|                 90|                 200|         6247500|         3998400|                  12.0| 41.666666666666664|             0.45|\n",
      "|100006|Беспроводные нал...|              1200|               1080|     12999|                 90|                 120|        19498500|        12478320|                   5.4| 55.555555555555564|             0.65|\n",
      "|100007|Умные часы с мони...|               400|                360|     29999|                 90|                  40|        14999500|         9599640|                   4.5| 22.222222222222225|             0.60|\n",
      "|100008|Кухонный комбайн...|               250|                225|     34999|                 90|                  25|        11549670|         6999720|                  2.25| 35.555555555555564|             0.50|\n",
      "|100009|Спортивный костю...|              1500|               1350|      5999|                 90|                 150|        11398100|         7198560|                   7.5| 53.333333333333336|             0.40|\n",
      "|100010|Электронная книга...|               180|                162|     89999|                 90|                  18|        21599760|         12959712|                   2.7| 22.222222222222225|             0.75|\n",
      "+------+--------------------+------------------+-------------------+----------+-------------------+--------------------+-----------------+------------------+----------------------+------------------+-----------------+\n",
      "only showing top 10 rows\n",
      "\n"
     ]
    }
   ],
   "source": [
    "# Выбор ключевых колонок для демонстрации\n",
    "result_columns = [\n",
    "    'sku_id', 'title', 'ordered_items_count', 'goods_sold_count', \n",
    "    'item_price', 'avg_percent_to_sold', 'returned_items_count', \n",
    "    'potential_revenue', 'total_revenue', 'avg_daily_sales', \n",
    "    'days_to_sold', 'item_rate_percent'\n",
    "]\n",
    "\n",
    "print(\"Результаты расчета дополнительных показателей (первые 10 строк):\")\n",
    "df_enriched.select(result_columns).show(10, truncate=True)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Статистика по расчетным показателям"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 8,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Статистика по расчетным показателям:\n",
      "+-------+---------------------+------------------+------------------+------------------+------------------+------------------+\n",
      "|summary|returned_items_count | potential_revenue|     total_revenue|    avg_daily_sales|       days_to_sold|  item_rate_percent|\n",
      "+-------+---------------------+------------------+------------------+------------------+------------------+------------------+\n",
      "|  count|               100000|            100000|            100000|            100000|            100000|            100000|\n",
      "|   mean|   152.34471999999998|2.550768881300001E7|1.2753534406500006E7| 11.55333333333333| 314.7416666666667|0.5000049999499999|\n",
      "| stddev|    145.1231472878585|2.468236876582001E7|1.2340984382910006E7|11.271666666666668| 768.9365079365079|0.2887091666666666|\n",
      "|    min|                    1|           30600.0|        -1349900.0|0.00819672131147541|               0.0|               0.0|\n",
      "|    max|                  755|         1.4859E8|         7.4295E7|             98.0|          12195.0|               1.0|\n",
      "+-------+---------------------+------------------+------------------+------------------+------------------+------------------+\n",
      "\n"
     ]
    }
   ],
   "source": [
    "calculated_columns = [\n",
    "    'returned_items_count', 'potential_revenue', 'total_revenue', \n",
    "    'avg_daily_sales', 'days_to_sold', 'item_rate_percent'\n",
    "]\n",
    "\n",
    "print(\"Статистика по расчетным показателям:\")\n",
    "df_enriched.select(calculated_columns).describe().show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Формирование финального отчета"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 9,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Финальный отчет сформирован\n",
      "Колонки в отчете:\n",
      "['sku_id', 'title', 'category', 'brand', 'seller', 'group_type', 'country', 'availability_items_count', 'ordered_items_count', 'warehouses_count', 'item_price', 'goods_sold_count', 'item_rate', 'days_on_sell', 'avg_percent_to_sold', 'returned_items_count', 'potential_revenue', 'total_revenue', 'avg_daily_sales', 'days_to_sold', 'item_rate_percent']\n"
     ]
    }
   ],
   "source": [
    "# Формирование финального отчета с требуемыми колонками\n",
    "final_columns = [\n",
    "    'sku_id', 'title', 'category', 'brand', 'seller', 'group_type', 'country',\n",
    "    'availability_items_count', 'ordered_items_count', 'warehouses_count',\n",
    "    'item_price', 'goods_sold_count', 'item_rate', 'days_on_sell',\n",
    "    'avg_percent_to_sold', 'returned_items_count', 'potential_revenue',\n",
    "    'total_revenue', 'avg_daily_sales', 'days_to_sold', 'item_rate_percent'\n",
    "]\n",
    "\n",
    "final_report = df_enriched.select(final_columns)\n",
    "\n",
    "print(\"Финальный отчет сформирован\")\n",
    "print(\"Колонки в отчете:\")\n",
    "print(final_report.columns)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Сохранение отчета в S3"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 10,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Отчет успешно сохранен в s3a://startde-project/roma_mesh/seller_items\n",
      "Формат: Parquet\n",
      "Режим записи: overwrite\n",
      "Количество партиций: 10\n"
     ]
    }
   ],
   "source": [
    "# Сохранение отчета в S3\n",
    "final_report.repartition(10) \\\n",
    "    .write \\\n",
    "    .mode(\"overwrite\") \\\n",
    "    .option(\"compression\", \"snappy\") \\\n",
    "    .parquet(OUTPUT_PATH)\n",
    "\n",
    "print(f\"Отчет успешно сохранен в {OUTPUT_PATH}\")\n",
    "print(\"Формат: Parquet\")\n",
    "print(\"Режим записи: overwrite\")\n",
    "print(\"Количество партиций: 10\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Валидация результатов"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 11,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Валидация расчетов:\n",
      "✓ returned_items_count: расчет корректен\n",
      "✓ potential_revenue: расчет корректен\n",
      "✓ total_revenue: расчет корректен\n",
      "✓ avg_daily_sales: расчет корректен\n",
      "✓ days_to_sold: расчет корректен\n",
      "✓ item_rate_percent: расчет корректен\n",
      "\n",
      "Проверка типов данных:\n",
      "root\n",
      " |-- sku_id: long (nullable = true)\n",
      " |-- title: string (nullable = true)\n",
      " |-- category: string (nullable = true)\n",
      " |-- brand: string (nullable = true)\n",
      " |-- seller: string (nullable = true)\n",
      " |-- group_type: string (nullable = true)\n",
      " |-- country: string (nullable = true)\n",
      " |-- availability_items_count: long (nullable = true)\n",
      " |-- ordered_items_count: long (nullable = true)\n",
      " |-- warehouses_count: long (nullable = true)\n",
      " |-- item_price: long (nullable = true)\n",
      " |-- goods_sold_count: long (nullable = true)\n",
      " |-- item_rate: double (nullable = true)\n",
      " |-- days_on_sell: long (nullable = true)\n",
      " |-- avg_percent_to_sold: long (nullable = true)\n",
      " |-- returned_items_count: integer (nullable = true)\n",
      " |-- potential_revenue: long (nullable = true)\n",
      " |-- total_revenue: long (nullable = true)\n",
      " |-- avg_daily_sales: double (nullable = true)\n",
      " |-- days_to_sold: double (nullable = true)\n",
      " |-- item_rate_percent: double (nullable = true)\n",
      "\n"
     ]
    }
   ],
   "source": [
    "print(\"Валидация расчетов:\")\n",
    "\n",
    "# Проверка корректности расчетов\n",
    "validation_sample = df_enriched.limit(100)\n",
    "\n",
    "# Проверка returned_items_count\n",
    "returned_check = validation_sample.filter(\n",
    "    col(\"returned_items_count\") == (col(\"ordered_items_count\") * (1 - col(\"avg_percent_to_sold\") / 100)).cast(IntegerType())\n",
    ).count() == 100\n",
    "print(f\"✓ returned_items_count: расчет {'корректен' if returned_check else 'некорректен'}\")\n",
    "\n",
    "# Проверка potential_revenue\n",
    "potential_check = validation_sample.filter(\n",
    "    col(\"potential_revenue\") == (col(\"availability_items_count\") + col(\"ordered_items_count\")) * col(\"item_price\")\n",
    ).count() == 100\n",
    "print(f\"✓ potential_revenue: расчет {'корректен' if potential_check else 'некорректен'}\")\n",
    "\n",
    "# Проверка total_revenue\n",
    "total_check = validation_sample.filter(\n",
    "    col(\"total_revenue\") == (col(\"goods_sold_count\") - col(\"returned_items_count\")) * col(\"item_price\")\n",
    ).count() == 100\n",
    "print(f\"✓ total_revenue: расчет {'корректен' if total_check else 'некорректен'}\")\n",
    "\n",
    "# Проверка avg_daily_sales\n",
    "avg_sales_check_count = validation_sample.filter(\n",
    "    (col(\"days_on_sell\") > 0) & (col(\"avg_daily_sales\") == col(\"goods_sold_count\") / col(\"days_on_sell\"))\n",
    ).count()\n",
    "zero_days_count = validation_sample.filter(col(\"days_on_sell\") == 0).count()\n",
    "avg_sales_check = (avg_sales_check_count + zero_days_count) == 100\n",
    "print(f\"✓ avg_daily_sales: расчет {'корректен' if avg_sales_check else 'некорректен'}\")\n",
    "\n",
    "# Проверка days_to_sold\n",
    "days_check_count = validation_sample.filter(\n",
    "    (col(\"avg_daily_sales\") > 0) & (col(\"days_to_sold\") == col(\"availability_items_count\") / col(\"avg_daily_sales\"))\n",
    ).count()\n",
    "zero_sales_count = validation_sample.filter(col(\"avg_daily_sales\") == 0).count()\n",
    "days_check = (days_check_count + zero_sales_count) == 100\n",
    "print(f\"✓ days_to_sold: расчет {'корректен' if days_check else 'некорректен'}\")\n",
    "\n",
    "# Проверка item_rate_percent\n",
    "percentile_stats = validation_sample.agg(\n",
    "    expr(\"min(item_rate_percent) as min_percent\"),\n",
    "    expr(\"max(item_rate_percent) as max_percent\")\n",
    ").collect()[0]\n",
    "percentile_check = (percentile_stats[0] >= 0) and (percentile_stats[1] <= 1)\n",
    "print(f\"✓ item_rate_percent: расчет {'корректен' if percentile_check else 'некорректен'}\")\n",
    "\n",
    "print(\"\\nПроверка типов данных:\")\n",
    "final_report.printSchema()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Завершение работы"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 12,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "=== ОТЧЕТ О ВЫПОЛНЕНИИ ПРОЕКТА ===\n",
      "\n",
      "✅ ОБРАБОТКА ДАННЫХ УСПЕШНО ЗАВЕРШЕНА\n",
      "\n",
      "📊 ОСНОВНЫЕ РЕЗУЛЬТАТЫ:\n",
      "- Обработано записей: 100,000\n",
      "- Добавлено расчетных полей: 6\n",
      "- Итоговых колонок в отчете: 21\n",
      "\n",
      "📈 КЛЮЧЕВЫЕ МЕТРИКИ:\n",
      "Среднее количество возвратов: 152.34 товаров\n",
      "Средняя потенциальная выручка: 25,507,689 ₽\n",
      "Средняя фактическая выручка: 12,753,534 ₽\n",
      "Средние дневные продажи: 11.55 товаров/день\n",
      "Среднее время до распродажи остатков: 314.74 дней\n",
      "\n",
      "💾 РЕЗУЛЬТАТЫ СОХРАНЕНЫ:\n",
      "S3 путь: s3a://startde-project/roma_mesh/seller_items\n",
      "Формат: Parquet (10 партиций)\n",
      "Сжатие: snappy\n",
      "\n",
      "🎯 ВСЕ РАСЧЕТЫ ПРОШЛИ ВАЛИДАЦИЮ\n",
      "\n",
      "Spark сессия остановлена\n"
     ]
    }
   ],
   "source": [
    "print(\"=== ОТЧЕТ О ВЫПОЛНЕНИИ ПРОЕКТА ===\\n\")\n",
    "print(\"✅ ОБРАБОТКА ДАННЫХ УСПЕШНО ЗАВЕРШЕНА\\n\")\n",
    "print(\"📊 ОСНОВНЫЕ РЕЗУЛЬТАТЫ:\")\n",
    "print(f\"- Обработано записей: {final_report.count():,}\")\n",
    "print(f\"- Добавлено расчетных полей: {len(calculated_columns)}\")\n",
    "print(f\"- Итоговых колонок в отчете: {len(final_columns)}\")\n",
    "print(\"\\n📈 КЛЮЧЕВЫЕ МЕТРИКИ:\")\n",
    "print(f\"Среднее количество возвратов: {df_enriched.agg({'returned_items_count': 'mean'}).collect()[0][0]:.2f} товаров\")\n",
    "print(f\"Средняя потенциальная выручка: {df_enriched.agg({'potential_revenue': 'mean'}).collect()[0][0]:,.0f} ₽\")\n",
    "print(f\"Средняя фактическая выручка: {df_enriched.agg({'total_revenue': 'mean'}).collect()[0][0]:,.0f} ₽\")\n",
    "print(f\"Средние дневные продажи: {df_enriched.agg({'avg_daily_sales': 'mean'}).collect()[0][0]:.2f} товаров/день\")\n",
    "print(f\"Среднее время до распродажи остатков: {df_enriched.agg({'days_to_sold': 'mean'}).collect()[0][0]:.2f} дней\")\n",
    "print(\"\\n💾 РЕЗУЛЬТАТЫ СОХРАНЕНЫ:\")\n",
    "print(f\"S3 путь: {OUTPUT_PATH}\")\n",
    "print(\"Формат: Parquet (10 партиций)\")\n",
    "print(\"Сжатие: snappy\")\n",
    "print(\"\\n🎯 ВСЕ РАСЧЕТЫ ПРОШЛИ ВАЛИДАЦИЮ\")\n",
    "\n",
    "# Остановка Spark сессии\n",
    "spark.stop()\n",
    "print(\"\\nSpark сессия остановлена\")"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "PySpark",
   "language": "python",
   "name": "pyspark"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.0"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}

SyntaxError: closing parenthesis ')' does not match opening parenthesis '[' on line 517 (459863634.py, line 526)